In [ ]:
# https://github.com/echocatzh/conv-stft

!pip install torch
!pip install onnx onnxruntime
!pip install openai-whisper
!pip install git+https://github.com/echocatzh/conv-stft.git

In [ ]:
import torch
from whisper.audio import mel_filters
from conv_stft import STFT

N_MELS = 80

audio = torch.randn(1, 16000*30)

class LogMelNet(torch.nn.Module):
    def __init__(self):
        super(LogMelNet, self).__init__()
        self.filters = mel_filters('cpu', N_MELS)
        self.logmelmodel = STFT(win_len=400, win_hop=160, fft_len=400)

    def forward(self, audio):
        stft = self.logmelmodel.transform(audio, return_type='magphase')[0]
        magnitudes = stft[..., :-1].abs() ** 2

        mel_spec = self.filters @ magnitudes

        log_spec = torch.clamp(mel_spec, min=1e-10).log10()
        log_spec = torch.maximum(log_spec, log_spec.max() - 8.0)
        log_spec = (log_spec + 4.0) / 4.0
        return log_spec

#logmel = whisper.log_mel_spectrogram(audio)

logmelmodel = LogMelNet()
torch.onnx.export(logmelmodel, (audio), "LogMelSepctro.onnx", export_params=True, do_constant_folding=True, input_names = ['audio'],  output_names=['log_mel'])#, dynamic_axes={'audio_input' : {1 : 'n_mels', 2 : 'n_ctx'}})